# This file included the complete pipeline without parallelization
Use single_run_patallel.py for parallelization

# Library Imports

In [ ]:
import importlib

import ipynb.fs.full.preprocessing0
import ipynb.fs.full.preprocessing05
#import ipynb.fs.full.prepossessing075_akistage
import preprocessing1
#import ipynb.fs.full.preprocessing2_BT
import preprocessing2_BT

import ipynb.fs.full.preprocessing25_BTcorr
import ipynb.fs.full.preprocessing3_smote
#import ipynb.fs.full.preprocessing4
import preprocessing4

#import ipynb.fs.full.runxgboost
import runxgboost

#import ipynb.fs.full.postprocessing1_SHAP

import postprocessing1_SHAP

import ipynb.fs.full.postprocessing3_collect

from ipynb.fs.full.slackbot import ping_slack
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
#import rpy2.robjects as robjects
#from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
from os.path import exists
import logging
import time

import utils_function

#Reload library to update on any change
#importlib.reload(ipynb.fs.full.preprocessing0)
importlib.reload(ipynb.fs.full.preprocessing05)
#importlib.reload(ipynb.fs.full.preapossessing075_akistage)
importlib.reload(preprocessing1)
importlib.reload(preprocessing2_BT)
importlib.reload(ipynb.fs.full.preprocessing25_BTcorr)
importlib.reload(ipynb.fs.full.preprocessing3_smote)
# #importlib.reload(ipynb.fs.full.preprocessing4)
importlib.reload(preprocessing4)
importlib.reload(runxgboost)
importlib.reload(postprocessing1_SHAP)
importlib.reload(ipynb.fs.full.postprocessing3_collect)
importlib.reload(postprocessing1_SHAP)
importlib.reload(utils_function)

# Data Requirement
Data should be in POCRNET CDM Format

# Generate Lab Loinc groups and Unit

This session unified lab measurements into unified units and group LOINC code into LOINC groups 

In [ ]:
# labunify.py
# Description: A class for unifying lab units and group LOINC code into LOINC groups.
# Output files: local_custom_convert.csv
# This file unify the lab units within single site
# Output files: UCUMqualX.csv
# This file cleans the quantitative labs
# Output files: UCUMunitX.csv
# This file unify the lab units globally
# Generated file to be used in preprocessing1.unify_lab

import labunify
labunify_obj = labunify.labunify()
labunify_obj.read_loinc()
labunify_obj.read_lab()
labunify_obj.get_all_relation()
labunify_obj.gen_local_conversion_table()
labunify_obj.local_custom_convert.to_csv('local_custom_convert.csv')
labunify_obj.get_consensus_unit()
labunify_obj.UCUMunitX.to_csv('UCUMunitX.csv')
labunify_obj.handle_qualitative()
labunify_obj.UCUMqualX.to_csv('UCUMqualX.csv')

# Generate RX to ATC conversion Table

This session convert all RX and NDC code to ATC code lv4

In [ ]:
# Cross-reference medunify.ipynb
# Run rxnorm2atcR_site.ipynb to translate at the end

# rxnorm2atcR_site.ipynb
# Description: This file convert RX and NDC code to ATC4th code 
# Output files: med_unified_conversion_rx2atc.parquet
# This files contain conversion from ndc to rx
# Output files: med_unified_conversion_nd2rx.parquet
# This files contain conversion from rx to ATC4th
# To be used in preprocessing1.amed

sites = ['UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UPITT', 'UTHSCSA', 'UNMC', 'KUMC']
configs_variable = utils_function.read_config('UTHSCSA')
site, datafolder, home_directory = utils_function.get_commons(configs_variable)

#Collect all med code in database
ameds = []
for site in sites:
    print(site)
    ameds.append(pd.read_parquet(datafolder+site+'/p0_amed_'+site+'.parquet')[['MEDADMIN_TYPE', 'MEDADMIN_CODE']].drop_duplicates())

amed = pd.concat(ameds).drop_duplicates()
    amed.to_parquet(datafolder+'/med_unified_conversion_before.parquet')
    
print("RUN rxnorm2atcR_site.ipynb")

# Preprocessing

This session prepare preprocess the data from multiple sites into a single table for machine learning

In [ ]:
# Read the config files for variables
# The location of config files is at '/home/hoyinchan/code/AKI_CDM_PY/configs_files/publish_config/Configs_init.txt'

site_list = ['KUMC', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UPITT', 'UTHSCSA', 'UNMC']
for site in site_list:
    configs_variables = utils_function.read_config(site, suffix='init', config_base_location='/lahome/hoyinchan/code/AKI_CDM_PY/configs_files/publish_config/')
    utils_function.write_config(configs_variables, suffix=site)
    configs_variables

# Change the sites name here    
configs_variable = utils_function.read_config('KUMC')

In [ ]:
# Unify Lab
preprocessing1.unify_lab(configs_variables)

# Drop outliner
ipynb.fs.full.preprocessing05.lab_drop_outliner(configs_variables)
ipynb.fs.full.preprocessing05.vital_drop_outliner(configs_variables)

# Split dataset into smaller files if needed for RAM consideration, using 'year' as index 
# Run parquet_splitter.py (reference: parquet_splitter.ipynb)

onset = pd.read_parquet(configs_variable['datafolder']+configs_variable['site']+'/p0_onset_'+configs_variable['site']+'.parquet')
years = list(pd.to_datetime(onset['ADMIT_DATE']).dt.year.unique())    

# Preprocess raw table into extracted table (per year)
for year in years:
    preprocessing1.demo(configs_variable,year)
    preprocessing1.vital(configs_variable,year)
    preprocessing1.dx(configs_variable,year)
    preprocessing1.px(configs_variable,year)
    preprocessing1.lab(configs_variable,year)            
    preprocessing1.amed(configs_variable,year)
    preprocessing2_BT.bigtable(configs_variable,year)

# Preprocess raw table into extracted table, drop missing data, set flag, rename column
preprocessing4.combinebtpos(configs_variable)

# GLOBAL (Abandoned)
# Collect ALL site, then use preprocessing25_BTcorr to remove correlated variables
# Generate site correlation
#ipynb.fs.full.preprocessing25_BTcorr.generate_corr(configs_variable)

# Calculate global corr and feature selection (run it in preprocessing25_BTcorr.ipynb
#ipynb.fs.full.preprocessing25_BTcorr.calculate_corr_occurence_new()

# Drop columns based on result
#ipynb.fs.full.preprocessing25_BTcorr.remove_correlated_features(configs_variable)

# Run CatBoost

In [ ]:
# Create test train split with different oversample strategy
ipynb.fs.full.preprocessing3_smote.generate_all_pre_catboost(configs_variable)

# Create Cross validation sets
ipynb.fs.full.preprocessing3_smote.gen_crossvalidate(configs_variable)

# Run single catboost
runxgboost.runxgboost(configs_variable)

# 10-fold cross validation
ipynb.fs.full.runxgboost.gen_crossvalidate(configs_variable)
for i in range(configs_variables['n_splits']):
    runxgboost.boosttrapcatboost(configs_variable, i)

# Postprocessing

Calculate the SHAP values for use in meta-analysis (Need to run every site CatBoost before running this code)

In [ ]:
sites = ['MCW', 'UIOWA', 'UMHC', 'UNMC', 'UofU', 'UTHSCSA', 'KUMC', 'UTSW', 'UPITT']

# External validation
for configs_variables_m in configs_variables:
    for configs_variables_d in configs_variables:
        for i in range(configs_variables['n_splits']):
            postprocessing1_SHAP.collectSHAP_cross_sub_validate(configs_variables_m, configs_variables_d, i)
            
# Collect SHAP raw for Metaregression
ipynb.fs.full.postprocessing3_collect.collect_collectSHAPraw_cross_sub(configs_variables)

# Meta-regression
Run MetaRegression/metagam3R.ipynb

# Interrupted Time series
Run timeseries/medication_its.ipynb and timeseries/medication_itsR.ipynb

# Confounding Analysis

Run timeries/confounder_regression_analysis.ipynb and Run timeries/confounder_regression_analysisR.ipynb


# Ploting script

For AUROC heatmap and feature importance plot, run plots_for_paper.ipynb

For demographic table, run gen_demo_table.ipynb

For flow chart, feature counts, run Produce_Cohort_Table2.ipynb

For Meta-regression plots, run plotmeta.ipynb

For Interrupted Time series plots, run timeries/medication_itsplot.ipynb and timeries/medication_itsplotR.ipynb

In [ ]:
# Turn to timeseries folder for timeseries analysis